<a href="https://colab.research.google.com/github/singhsneha99/Weather-vs-Crime-data/blob/main/RBDApart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib==3.1.0

     |████████████████████████████████| 13.1 MB 4.6 MB/s 
ERROR: Operation cancelled by user


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics, preprocessing, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from math import sqrt
import os, sys, itertools, re
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
%matplotlib inline


In [ ]:
if 'google.colab' in sys.modules:    
    project_path = "/content/drive/My Drive/"
    # Google Colab lib
    from google.colab import drive
    # Mount the drive
    drive.mount('/content/drive/', force_remount=True)
    sys.path.append(project_path)
    %cd $project_path

# Let's look at the sys path
print('Current working directory', os.getcwd())

Mounted at /content/drive/
/content/drive/My Drive
Current working directory /content/drive/My Drive


In [ ]:
datatran_2007_df = pd.read_csv('por_ocorrencia/datatran2007.csv', sep = ';',encoding = 'latin-1')
datatran_2008_df = pd.read_csv('por_ocorrencia/datatran2008.csv', sep = ';',encoding = 'latin-1')
datatran_2009_df = pd.read_csv('por_ocorrencia/datatran2009.csv', sep = ';',encoding = 'latin-1')
datatran_2010_df = pd.read_csv('por_ocorrencia/datatran2010.csv', sep = ';',encoding = 'latin-1')
datatran_2011_df = pd.read_csv('por_ocorrencia/datatran2011.csv', sep = ';',encoding = 'latin-1')
datatran_2012_df = pd.read_csv('por_ocorrencia/datatran2012.csv', sep = ';',encoding = 'latin-1')
datatran_2013_df = pd.read_csv('por_ocorrencia/datatran2013.csv', sep = ';',encoding = 'latin-1')
datatran_2014_df = pd.read_csv('por_ocorrencia/datatran2014.csv', sep = ';',encoding = 'latin-1')
datatran_2015_df = pd.read_csv('por_ocorrencia/datatran2015.csv', sep = ';',encoding = 'latin-1')
datatran_2016_df = pd.read_csv('por_ocorrencia/datatran2016.csv', sep = ';',encoding = 'latin-1')
datatran_2017_df = pd.read_csv('por_ocorrencia/datatran2017.csv', sep = ';',encoding = 'latin-1')
datatran_2018_df = pd.read_csv('por_ocorrencia/datatran2018.csv', sep = ';',encoding = 'latin-1')
datatran_2019_df = pd.read_csv('por_ocorrencia/datatran2019.csv', sep = ';',encoding = 'latin-1')
datatran_2020_df = pd.read_csv('por_ocorrencia/datatran2020.csv', sep = ';',encoding = 'latin-1')
datatran_2021_df = pd.read_csv('por_ocorrencia/datatran2021.csv', sep = ';',encoding = 'latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
datatran_df = pd.concat([datatran_2007_df,datatran_2008_df,datatran_2009_df,datatran_2010_df,datatran_2011_df,
                         datatran_2012_df,datatran_2013_df,datatran_2014_df,datatran_2015_df,datatran_2016_df,
                         datatran_2017_df,datatran_2018_df,datatran_2019_df, datatran_2020_df,datatran_2021_df])

In [ ]:
datatran_df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,10.0,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,0,5,0,4,6,NaN,NaN,NaN,NaN,NaN
1,10.0,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,0,5,0,4,6,NaN,NaN,NaN,NaN,NaN
2,1032898.0,13/08/2007,Segunda,14:25:00,MG,40,585.5,ITABIRITO,Outras,Saída de Pista,...,1,2,0,1,1,NaN,NaN,NaN,NaN,NaN
3,1051130.0,12/02/2007,Segunda,02:10:00,MA,135,11.0,SAO LUIS,Animais na Pista,Atropelamento de animal,...,1,0,0,3,1,NaN,NaN,NaN,NaN,NaN
4,1066824.0,20/11/2007,Terça,05:30:00,CE,222,30.8,CAUCAIA,Defeito mecânico em veículo,Capotamento,...,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN


## Profiling on imported data


In [ ]:
datatran_df.shape

(1882953, 31)

In [ ]:
#BASIC DATA PROFILING
datatran_df.describe()

,id,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
count,1.882953e+06,1.465837e+06,1.882953e+06,1.882953e+06,1.882953e+06,1.882953e+06,1.882953e+06,1.882953e+06,1.882953e+06,1.882953e+06
mean,2.131641e+07,2.011082e+03,2.187296e+00,5.390204e-02,5.120648e-01,1.789306e-01,1.347950e+00,9.439800e-02,6.909955e-01,1.725757e+00
std,3.580286e+07,2.412219e+00,1.613012e+00,2.887796e-01,1.004068e+00,5.693595e-01,1.268478e+00,3.892005e-01,1.203814e+00,7.265909e-01
min,8.000000e+00,2.007000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,3.583710e+05,2.009000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,8.429990e+05,2.011000e+03,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
75%,8.303990e+07,2.013000e+03,2.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
max,8.352989e+07,2.015000e+03,2.480000e+02,3.300000e+01,5.800000e+01,2.220000e+02,9.900000e+01,8.600000e+01,2.390000e+02,2.500000e+01


In [ ]:
#datatran_df.to_csv('df.csv')
data= datatran_df

In [ ]:
data.isnull().sum() #check for missing values in the dataset

id                              0
data_inversa                    0
dia_semana                      0
horario                         0
uf                              0
br                            597
km                            597
municipio                       0
causa_acidente                  0
tipo_acidente                   0
classificacao_acidente          1
fase_dia                        1
sentido_via                     0
condicao_metereologica          3
tipo_pista                      0
tracado_via                     0
uso_solo                        0
ano                        417116
pessoas                         0
mortos                          0
feridos_leves                   0
feridos_graves                  0
ilesos                          0
ignorados                       0
feridos                         0
veiculos                        0
latitude                  1562200
longitude                 1562200
regional                  1562200
delegacia     

In [ ]:
data.drop(['latitude', 'longitude','regional','delegacia','uop'], axis=1,inplace=True)

In [ ]:
data.drop(['ano'],axis=1)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,10.0,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,Reta,Rural,9,0,4,0,5,0,4,6
1,10.0,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,Reta,Rural,9,0,4,0,5,0,4,6
2,1032898.0,13/08/2007,Segunda,14:25:00,MG,40,585.5,ITABIRITO,Outras,Saída de Pista,...,Reta,Rural,3,0,0,1,2,0,1,1
3,1051130.0,12/02/2007,Segunda,02:10:00,MA,135,11.0,SAO LUIS,Animais na Pista,Atropelamento de animal,...,Reta,Urbano,5,2,2,1,0,0,3,1
4,1066824.0,20/11/2007,Terça,05:30:00,CE,222,30.8,CAUCAIA,Defeito mecânico em veículo,Capotamento,...,Reta,Rural,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30896,373974.0,2021-06-16,quarta-feira,13:15:00,BA,116.0,466,SANTO ESTEVAO,Condutor deixou de manter distância do veículo...,Colisão traseira,...,Reta,Não,2,0,1,0,0,1,1,2
30897,374024.0,2021-06-28,segunda-feira,10:50:00,PR,277.0,576,CASCAVEL,Velocidade Incompatível,Engavetamento,...,Reta,Não,8,0,2,0,6,0,2,5
30898,374224.0,2021-06-13,domingo,02:50:00,SP,116.0,"102,7",PINDAMONHANGABA,Reação tardia ou ineficiente do condutor,Colisão traseira,...,Ponte,Não,2,0,1,0,0,1,1,2
30899,374239.0,2021-06-02,quarta-feira,08:45:00,RN,101.0,85,NATAL,Velocidade Incompatível,Colisão traseira,...,Curva,Sim,2,0,0,1,1,0,1,2


In [ ]:
data['br'].fillna(data['br'].mean)

0          381
1          381
2           40
3          135
4          222
         ...  
30896    116.0
30897    277.0
30898    116.0
30899    101.0
30900    101.0
Name: br, Length: 1882953, dtype: object

In [ ]:
data['km'].fillna(data['km'].mean)

0        623.2
1        623.2
2        585.5
3         11.0
4         30.8
         ...  
30896      466
30897      576
30898    102,7
30899       85
30900     78,4
Name: km, Length: 1882953, dtype: object

In [ ]:
import pandas as pd
df_en = data.copy()

In [ ]:
# install googletrans using pip
!pip install googletrans



In [ ]:
# import the library
import googletrans
from googletrans import Translator


In [ ]:
translator = Translator()


In [ ]:
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [ ]:
df_col=data['causa_acidente'].unique()
df_col

array(['Falta de atenção', 'Outras', 'Animais na Pista',
       'Defeito mecânico em veículo',
       'Não guardar distância de segurança', 'Velocidade incompatível',
       'Desobediência à sinalização', 'Ingestão de álcool',
       'Defeito na via', 'Dormindo', 'Ultrapassagem indevida', '(null)',
       'Fenômenos da Natureza', 'Falta de Atenção à Condução',
       'Avarias e/ou desgaste excessivo no pneu', 'Ingestão de Álcool',
       'Defeito Mecânico no Veículo',
       'Desobediência às normas de trânsito pelo condutor',
       'Velocidade Incompatível', 'Restrição de Visibilidade',
       'Falta de Atenção do Pedestre', 'Condutor Dormindo',
       'Pista Escorregadia', 'Ultrapassagem Indevida', 'Defeito na Via',
       'Sinalização da via insuficiente ou inadequada', 'Mal Súbito',
       'Carga excessiva e/ou mal acondicionada',
       'Objeto estático sobre o leito carroçável',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Inge

In [ ]:
df_en = translator.translate(df_col, dest='en')
df_en.text

"['Lack of attention' 'Other' 'Animals on the Track'\n 'Mechanical defect in vehicle' 'Do not keep safety distance'\n 'Incompatible speed' 'Disobeying the signal'\n 'Alcohol ingestion' 'Defect in the road' 'Sleeping' 'Improper overtaking'\n '(null)' 'Phenomena of Nature' 'Lack of Attention to Driving'\n 'Breakdowns and/or excessive wear on the tire' 'Alcohol ingestion'\n 'Vehicle Mechanical Defect'\n 'Disobedience to traffic rules by the driver'\n 'Incompatible Speed' 'Visibility Restriction'\n 'Lack of Attention by Pedestrian' 'Driver Sleeping' 'Slippery Road'\n 'Improper Overtaking' 'Defect in the Road'\n 'Insufficient or inadequate road signage' 'Mal Subito'\n 'Excessive and/or poorly packaged load'\n 'Static object on the carriage bed'\n 'Deficiency or non-activation of the Vehicle Lighting/Signalling System'\n 'Ingestion of Psychoactive Substances' 'External Aggression'\n 'Disobedience to traffic rules by pedestrians'\n 'Ingestion of alcohol and/or psychoactive substances by pedes

In [ ]:
pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 361 kB/s 
     |████████████████████████████████| 42 kB 567 kB/s 
     |████████████████████████████████| 1.3 MB 14.0 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=89b62a06efc340b5200970c439933b44ed29898ba0bf353fedc35c992ef49d90
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
